In [ ]:
import pathlib
import itertools
import functools

import numpy as np
import matplotlib.pyplot as plt

from IPython import display

import pydicom

from pymedphys._dicom import dose as pmp_dose
from pymedphys._dicom import rtplan as pmp_plan

In [ ]:
@functools.lru_cache()
def load_dicom_file(path):
    return pydicom.read_file(path, force=True)

In [ ]:
DATA_ROOT = pathlib.Path(r'S:\Physics\DoseCHECK\Commissioning\MLC test field Results\DICOM Comparisons\DICOM Exports')

In [ ]:
systems = [
    path.name for path in DATA_ROOT.glob('*')
]

systems

In [ ]:
dose_check = DATA_ROOT.joinpath('DoseCHECK')
models = [
    path.name for path in dose_check.glob('*')
]
models

In [ ]:
energies_to_check = {'06FFF', '10FFF'}
fields_to_check = {'3ABUT', '7SegA', '10x10'}

plans_to_check = {
    (energy, field): f'MC{energy}_{field}'
    for energy, field in itertools.product(energies_to_check, fields_to_check)
}

plans_to_check

In [ ]:
monaco_paths = {
    key: DATA_ROOT.joinpath('Monaco', plan_name, 'dose.dcm')
    for key, plan_name in plans_to_check.items()
}

monaco_paths

In [ ]:
plan_files = {
    key: DATA_ROOT.joinpath('Monaco', plan_name, 'plan.dcm')
    for key, plan_name in plans_to_check.items()
}

plan_files

In [ ]:
dosecheck_paths = {
    key: {
        model_name: DATA_ROOT.joinpath('DoseCHECK', model_name, plan_name, 'dose.dcm')
        for model_name in models
    }
    for key, plan_name in plans_to_check.items()
}

dosecheck_paths

In [ ]:
depth_lookup = 100

def plot_crossplane_profile_at_depth(dose_path, plan_path, depth_lookup, label):
    try:
        dcm_dose = load_dicom_file(dose_path)
    except FileNotFoundError as e:
        print(e)
        return
    
    dcm_plan = load_dicom_file(plan_path)
    
    surface_point = pmp_plan.get_surface_entry_point_with_fallback(dcm_plan)
    depth_adjust = surface_point.y
    y_lookup = depth_lookup + depth_adjust
    
    (z, y, x), dose = pymedphys.dicom.zyx_and_dose_from_dataset(dcm_dose)

    depth_index = np.where(y==y_lookup)[0][0]
    z_index = np.where(z==0)[0][0]
    
    plt.plot(x, dose[z_index, depth_index, :], label=label)


for key, monaco_path in monaco_paths.items():
    energy = key[0]
    plan_name = key[1]
    display.display(display.Markdown(f"## {energy} | {plan_name}"))
    
    print(monaco_path)
    
    plt.figure(figsize=(12,5))    
    plot_crossplane_profile_at_depth(monaco_path, plan_files[key], depth_lookup, "Monaco")

    
    for model_name, dose_check_path in dosecheck_paths[key].items():
        print(dose_check_path)
        plot_crossplane_profile_at_depth(dose_check_path, plan_files[key], depth_lookup, model_name)
    
    plt.xlim([-100, 100])
    plt.legend()
    plt.show()